In [1]:
from sportstat.database import init_db, session
from sportstat.models import Team, Athlete, Game, Play, Action, Observation
#__dbname__ = 'sqlite:///./tmp/test.db'
#init_db()  # __dbname__ not defined; using in-memory sqlite
path_to_data_dir = '../data'

In [2]:
#make a new game between alamaba and osu
osu = session.query(Team).filter_by(name='Ohio State').first()
bama = session.query(Team).filter_by(name='Alabama').first()
game = Game()
game.home_team_id = osu.id
game.away_team_id = bama.id
session.add(game)
session.commit()
print(game.id)

1


In [3]:
#parse play data

decode_def_off = {
    'OSU': osu,
    'BAMA': bama
}

file = open(path_to_data_dir + '/sugar_bowl.csv','r')
lines = file.readlines()
for line in lines[2:]:#first two lines are meta
    cols = line.split(',')
    if cols[0].strip() != '':
        play_index = cols[0].strip()
        offense_code = cols[1].strip()
        print(offense_code)
        defense_code = cols[2].strip()
        offense = decode_def_off[offense_code]
        defense = decode_def_off[defense_code]
        quarter = int(cols[3].strip())
        down = int(cols[4].strip())
        print(offense.id)
        #yards_to_go = int(cols[5].strip())
        half = cols[6].strip()
        yards = cols[7].strip()
        result = cols[8].strip()
        
        play = Play()
        play.game_id = game.id
        play.play_index = int(play_index)
        play.quarter_index = int(quarter)
        play.offense_id = offense.id
        play.defense_id = defense.id
        play.down = down
        #play.yards_to_go = yards_to_go
        play.half = bool(half == 'Own')
        play.yards = int(yards)
        play.result = result
        
        session.add(play)
session.commit()
file.close()

BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
OSU
35
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
BAMA
104
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
OSU
35
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104
BAMA
104


In [4]:
#get quarterback actions
file = open(path_to_data_dir + '/sugar_bowl.csv','r')
lines = file.readlines()
for line in lines[2:]:#first two lines are meta
    cols = line.split(',')
    if cols[0].strip() != '':
        play_index = cols[0].strip()
        game_id = 1
        play_id = session.query(Play)\
            .join(Game)\
            .filter(Play.play_index == play_index)\
            .filter(Game.id == game_id)\
            .one().id
        offense_code = cols[1].strip()
        offense = decode_def_off[offense_code]
    quarterback = cols[9].strip()
    if quarterback != '':
        first_name, last_name, number = quarterback.split(' ')
        first_name = first_name.strip()
        last_name = last_name.strip()
        sguc = cols[10].strip()
        alt = cols[11].strip()
        comp = cols[12].strip()
        direction = cols[13].strip()
        air_yards = cols[14].strip()
        total_yards = cols[15].strip()
        press = cols[16].strip()
        pock = cols[17].strip()
        scram = cols[18].strip()
        
        qb = session.query(Athlete).filter(Athlete.position == 'QB')
        qb = qb.filter(Athlete.first_name == first_name)
        qb = qb.filter(Athlete.last_name == last_name)
        try:
            qb = qb.one()
        except:
            print('no matching qb for ' + first_name + ' ' + last_name)
            #placeholder = Athlete
            #placeholder.team_id = offense.id
            #placeholder.position = 'QB'
            #placeholder.first_name = first_name
            #placeholder.last_name = last_name
        else:
            qb_action = Action()
            qb_action.play_id = play_id
            qb_action.athlete_id = qb.id
            qb_action.action_type = 'QB'
            session.add(qb_action)
session.commit()
        
file.close()

no matching qb for Evan Spencer


In [5]:
qbs = session.query(Athlete)
qbs = qbs.filter(Athlete.last_name == 'Spencer')
                            
for qb in qbs:
    print(qb.first_name + ' ' + qb.last_name)

Domenic Spencer
Jalen Spencer
Joe Spencer
Colin Spencer
Trey Spencer
Isaiah Spencer
Sean Spencer
Justin Spencer
Octavius Spencer
Blake Spencer
Jameel Spencer


In [6]:
session.close()

In [7]:
qb = session.query(Athlete).filter(Athlete.position == 'QB')
qb = qb.filter(Athlete.first_name == 'Blake')
qb = qb.filter(Athlete.last_name == 'Sims')
qb = qb.first()
print(qb.team.name)

Alabama


In [8]:
session.query(Play)\
    .join(Game)\
    .filter(Play.play_index == play_index)\
    .filter(Game.id == 1)\
    .one().id

108

In [9]:
for play in session.query(Play):
    print(play.yards)

25
30
32
15
19
19
31
44
41
5
5
5
12
18
17
20
33
25
25
36
36
38
42
42
41
1
9
4
21
47
45
45
36
29
25
15
12
14
2
39
46
46
5
14
15
15
15
29
31
15
11
6
6
2
29
30
29
45
50
44
44
18
18
5
6
5
6
3
10
16
18
23
37
42
40
13
13
25
36
44
49
49
47
19
37
45
39
5
9
15
22
22
34
41
43
1
2
12
33
37
36
16
16
31
17
10
5
5
